# 로지스틱 회귀 상관성 분석

In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1. CSV 파일 불러오기
df = pd.read_csv("분석용데이터_소득생활인구_아동인구_보간완료.csv")

# 2. 평균값 변수 생성
df['총생활인구_평균'] = df[['총생활인구_2024Q1', '총생활인구_2024Q2', '총생활인구_2024Q3', '총생활인구_2024Q4']].mean(axis=1)
df['소득_평균'] = df[['소득_2024Q1', '소득_2024Q2', '소득_2024Q3', '소득_2024Q4']].mean(axis=1)
df['아동인구_평균'] = df[
    ['아동인구_0_4세_2024Q1', '아동인구_0_4세_2024Q2', '아동인구_0_4세_2024Q3', '아동인구_0_4세_2024Q4',
     '아동인구_5_9세_2024Q1', '아동인구_5_9세_2024Q2', '아동인구_5_9세_2024Q3', '아동인구_5_9세_2024Q4']
].mean(axis=1)

# 3. 로그변환 컬럼 생성
df['총생활인구_평균_log'] = np.log1p(df['총생활인구_평균'])
df['소득_평균_log'] = np.log1p(df['소득_평균'])
df['아동인구_평균_log'] = np.log1p(df['아동인구_평균'])

# 교통, 보육 통합 변수
df["대중교통_수"] = df["버스정류장_수"] + df["지하철역_수"]
df["보육시설_수"] = df["유치원_수"] + df["어린이집_수"]

# 4. 회귀 분석용 변수 정의
cols_logit = [
    "총생활인구_평균_log", "아동인구_평균_log", "소득_평균_log", "대중교통_수",
    "보육시설_수", "초등학교_수", "키움센터_개수", "유흥시설_수", "일반키즈카페_개수"
]

X = df[cols_logit]
X = sm.add_constant(X)  # 절편항 추가
y = (df["서울형키즈카페_개수"] > 0).astype(int)  # 서울형 키즈카페 유무 (0 또는 1)

# 5. 로지스틱 회귀 모델 적합
logit_model = sm.Logit(y, X)
result = logit_model.fit(disp=0)

# 6. 결과 요약 출력
summary_df = result.summary2().tables[1].reset_index()
summary_df.columns = ["변수명", "회귀계수", "표준오차", "z", "p-value", "신뢰구간_하", "신뢰구간_상"]

# 7. 출력
print(summary_df)

            변수명      회귀계수      표준오차         z   p-value    신뢰구간_하     신뢰구간_상
0         const  8.532795  7.624361  1.119149  0.263077 -6.410678  23.476267
1  총생활인구_평균_log  0.254848  0.402694  0.632856  0.526827 -0.534418   1.044114
2   아동인구_평균_log  0.368570  0.368735  0.999552  0.317528 -0.354138   1.091277
3     소득_평균_log -1.032627  0.482264 -2.141209  0.032257 -1.977846  -0.087408
4        대중교통_수 -0.000858  0.004784 -0.179289  0.857711 -0.010234   0.008518
5        보육시설_수 -0.028785  0.034601 -0.831915  0.405457 -0.096603   0.039032
6        초등학교_수  0.173876  0.128035  1.358031  0.174454 -0.077069   0.424820
7       키움센터_개수  0.004534  0.003471  1.306001  0.191552 -0.002270   0.011337
8        유흥시설_수 -0.004583  0.004166 -1.100052  0.271309 -0.012749   0.003583
9     일반키즈카페_개수  0.040121  0.118810  0.337694  0.735594 -0.192741   0.272984


# 라쏘 회귀 분석

In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

# 1. 데이터 불러오기
df = pd.read_csv("분석용데이터_소득생활인구_아동인구_보간완료.csv")

# 2. 파생변수 생성
df["대중교통_수"] = df["버스정류장_수"] + df["지하철역_수"]
df["보육시설_수"] = df["어린이집_수"] + df["유치원_수"]

# 3. 평균 컬럼 생성 (2024Q1~Q4 기준)
df["총생활인구_평균"] = df[[f"총생활인구_2024Q{i}" for i in range(1, 5)]].mean(axis=1)
df["소득_평균"] = df[[f"소득_2024Q{i}" for i in range(1, 5)]].mean(axis=1)
df["아동인구_평균"] = df[[f"아동인구_0_4세_2024Q{i}" for i in range(1, 5)] + 
                       [f"아동인구_5_9세_2024Q{i}" for i in range(1, 5)]].mean(axis=1)

# 4. 로그변환
df["총생활인구_평균_log"] = np.log1p(df["총생활인구_평균"])
df["소득_평균_log"] = np.log1p(df["소득_평균"])
df["아동인구_평균_log"] = np.log1p(df["아동인구_평균"])

# 5. 회귀용 변수 선택
cols_lasso = [
    "총생활인구_평균_log", "아동인구_평균_log", "소득_평균_log", "대중교통_수",
    "보육시설_수", "초등학교_수", "키움센터_개수", "유흥시설_수", "일반키즈카페_개수"
]

X = df[cols_lasso]
y = df["서울형키즈카페_개수"]

# 6. 표준화 (Z-score)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 7. LASSO 회귀 모델 적합
lasso = Lasso(alpha=0.01)
lasso.fit(X_scaled, y)

# 8. 결과 출력
coefficients = pd.Series(lasso.coef_, index=cols_lasso)
print("LASSO 회귀 계수 (alpha=0.01):")
print(coefficients)

LASSO 회귀 계수 (alpha=0.01):
총생활인구_평균_log    0.023837
아동인구_평균_log     0.000490
소득_평균_log      -0.050434
대중교통_수         -0.005461
보육시설_수          0.000000
초등학교_수          0.011513
키움센터_개수         0.018582
유흥시설_수         -0.043345
일반키즈카페_개수       0.086469
dtype: float64


# 정준상관분석

In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import CCA

# 독립변수 (입지 요인 그룹)
X = df[[
    "총생활인구_log", "소득_log", "아동인구_평균_log", "보육시설_수",
    "초등학교_수", "대중교통_수", "유흥시설_수", "키움센터_개수", "일반키즈카페_개수"
]]

# 결과변수 그룹 정의
Y_서울형 = df[["서울형키즈카페_개수"]]
Y_공공형 = df[["서울형키즈카페_개수", "민간인증제키즈카페_개수"]].sum(axis=1).to_frame(name="공공형키즈카페_개수")

# 정규화
scaler_x = StandardScaler()
X_scaled = scaler_x.fit_transform(X)

# (1) 서울형 모델
scaler_y1 = StandardScaler()
Y1_scaled = scaler_y1.fit_transform(Y_서울형)

cca1 = CCA(n_components=1)
cca1.fit(X_scaled, Y1_scaled)
U1, V1 = cca1.transform(X_scaled, Y1_scaled)

# (2) 공공형 모델
scaler_y2 = StandardScaler()
Y2_scaled = scaler_y2.fit_transform(Y_공공형)

cca2 = CCA(n_components=1)
cca2.fit(X_scaled, Y2_scaled)
U2, V2 = cca2.transform(X_scaled, Y2_scaled)

# 7. 결과 출력
from scipy.stats import pearsonr

corr1 = pearsonr(U1.ravel(), V1.ravel())[0]
corr2 = pearsonr(U2.ravel(), V2.ravel())[0]

print("[서울형 키즈카페] 정준상관계수:", round(corr1, 4))
print("변수별 기여계수:")
for var, coef in zip(X.columns, cca1.x_weights_.ravel()):
    print(f" - {var}: {coef:.4f}")

print("\n[공공형 키즈카페] 정준상관계수:", round(corr2, 4))
print("변수별 기여계수:")
for var, coef in zip(X.columns, cca2.x_weights_.ravel()):
    print(f" - {var}: {coef:.4f}")

[서울형 키즈카페] 정준상관계수: 0.2447
변수별 기여계수:
 - 총생활인구_log: 0.2773
 - 소득_log: -0.4767
 - 아동인구_평균_log: 0.0010
 - 보육시설_수: -0.0251
 - 초등학교_수: 0.1183
 - 대중교통_수: -0.1337
 - 유흥시설_수: -0.3416
 - 키움센터_개수: 0.2292
 - 일반키즈카페_개수: 0.7029

[공공형 키즈카페] 정준상관계수: 0.61
변수별 기여계수:
 - 총생활인구_log: 0.0174
 - 소득_log: -0.1611
 - 아동인구_평균_log: -0.0764
 - 보육시설_수: 0.0597
 - 초등학교_수: 0.1397
 - 대중교통_수: 0.0515
 - 유흥시설_수: -0.1029
 - 키움센터_개수: 0.0486
 - 일반키즈카페_개수: 0.9640


# 